# Bioethanol Case Study - Fall 2024

Completed by Dallin Dunn, CJ Alvez, Justin Labrecque, and Patrick van Duyse as part of CH EN 273 - Chemical Process Principles

### Initialization

In [1]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad
# from myengineerpkg import molecular_weights as mw # an external script/package previously written by Patrick

### Abbreviations

- EtOH: Ethanol
- H2O: Water
- E199: 199-Proof Ethanol
- mash1: Mash after mixing tank and before thin stillage is added
- mash2: Final Mash before entering reactors 
- Other abbreviations taken from [Zybooks](https://learn.zybooks.com/zybook/BYUCHEN273KnottsFall2024/chapter/10/section/2)

### Properties

In [2]:
# ideal gas constant
R = 0.730240507295273 # ft^3 * atm / lbmol / R0.730240507295273

# STP
T_R__STP = 273.15 * 9 / 5 # degrees Rankine
P_atm__STP = 1 # atm

# mash1
SG_mash1 = 1.1 # relative to water

# bushels
m_corn__bushel = 56 # lbm corn per bushel

# 199-Proof Ethanol
SG_E199 = 0.7962 # at 60 F, relative to water
Xvol_EtOH__E199 = 0.9950 # % EtOH in E199 by vol
Xwt_EtOH__E199 = 0.9919 # % EtOH in E199 by weight
Xmol_EtOH__E199 = 0.9795 # % EtOH in E199 by mole

# Ethanol (EtOH)
# MW_EtOH = mw.by_formula("C2H5OH") # (g / mol) or (lbm / lbmol)
MW_EtOH = 46.069 # (g / mol) or (lbm / lbmol)

# Carbon Dioxide (CO2)
# MW_CO2 = mw.by_formula("CO2") # (g / mol) or (lbm / lbmol)
MW_CO2 = 44.009 # (g / mol) or (lbm / lbmol)

# Water (H2O)
ρ_H2O__4C = 8.33 # lbm / gal for water @ 4C, used for SG calculations
# MW_H2O = mw.by_formula("H2O") # (g / mol) or (lbm / lbmol)
MW_H2O = 18.015 # (g / mol) or (lbm / lbmol)

# reactor tanks
V_tank = 550000 # gal capacity / tank

# units
GAL_PER_FT3 = 7.48051948

### Specifications

In [3]:
# yields and flows per bushel
V_mash1__bushel = 22 # gal mash1 per bushel
V_mash2__bushel = 25 # gal mash2 per bushel
V_E199__bushel = 2.57 # gal E199 produced per bushel

# reactors
tank_fill = 0.9 # vol% mash2 / capacity
tank_count = 8 # tanks / cycle
T__tank = 8 # hrs / tank

# plant uptime
T_uptime__annual = 330*24 # hrs / year (330 days -> hrs)

### Problem 1 (Medium to High)

> Construct a process flow diagram of the process and a separate diagram for one of the fermentation batteries containing two reaction vessels, one pump, and one heat exchanger. Calculate and fill in the component flow rates $\left(\frac{lb_m}{hr}\right)$ and temperatures of each stream on the chart to the extent possible from the given information.

**Previously submitted. See Appendix 1**

### Problem 2 (Low)

> Operation of the fermentation reactors in a batch mode adds considerable complexity to the process. To assist with estimating the personnel requirements and options for automatic control for the fermentation plant, develop a schedule for switching the operation of the eight fermenters to provide for continuous feed to the section and appropriate circulation of mash through the heat exchangers. The schedule should identify 8-hr periods for each fermenter; the periods are for the fermenter being fed, emptied, operated adiabatically, and operated with cooling.

#### Logic
Since the cooling pump only allows us to cool one tank from each set at a time, the paired tanks need to be staggered. Staggering them by a single 8-hour period allows us to keep the same pattern for all tanks, filling one at a time, with 48 hours of reaction time, 24 hours of which are cooling for each tank.

Having the 48-hr reaction period alternate between cooling operation and adiabatic operation allows for the temperature to stay as close to the desired temperature range as possible. In the final cooling period (right before emptying the tanks) the tank would ideally only be cooled to 90 F, allowing the intermediate products to enter the storage tank at the same temperature as the tank. This allows the storage tank to be steady-state (as far as temperature is concerned) despite the variable volume.

#### Schedule
|Unit|  0 |  8 | 16 | 24 | 32 | 40 | 48 | 56 |
|----|----|----|----|----|----|----|----|----|
|  1 |  F |  A |  C |  A |  C |  A |  C |  E |
|  2 |  E |  F |  A |  C |  A |  C |  A |  C |

#### Legend
- F: Filling
- A: Operating Adiabatically
- C: Operating while Cooling
- E: Emptying

This can be repeated for the other 3 reactor pairs, with a 16-hr stagger between pairs, and a total of 8 tanks filled over 64 hours.

### Problem 3 (Low)

> Calculate the feed rate of corn in $\frac{\text{lb}_m}{\text{hr}}$. Estimate the acreage required to supply this plant with corn.

#### Logic
Since we know how much final mash we need to fill the reaction tanks, and we know how frequently the tanks are filled, figuring out the mass flow rate of the final mash is trivial. We are also given the ratio of final mash to bushels, and the weight of corn per bushel, allowing for some trivial calculations to calculate the feed rate of the corn.

As for the acreage, based on [statistics from the USDA](https://www.nass.usda.gov/Charts_and_Maps/Crops_County/cr-yi.php), the annual corn harvest per acre in Iowa looks to be either 175-209.9 bushels or 210+ bushels, depending on the county. We will use 175 bushels in our calculations to adjust for harvest fluctuation. Calculating how many days/year the plant is active yields the active hours per year, allowing us to calculate the annual bushel demand and the consequent acreage requirement.

#### Calculations

In [4]:
# final mash calculations
V_mash2__tank = V_tank * tank_fill # gal mash2 / tank
V̇_mash2__cycle = V_mash2__tank * tank_count # gal mash2 / cycle
T__cycle = T__tank * tank_count # hrs / cycle

V̇_mash2 = V̇_mash2__cycle / T__cycle # gal mash2 / hr
print(rf"V̇_mash2 = {V̇_mash2:.0f} gal/hr")

# bushel calculations
ṅ_bushel__in = V̇_mash2/V_mash2__bushel # # bushels / hr
print(rf"ṅ_bushel__in = {ṅ_bushel__in:.0f} bushels/hr")

# corn feed rate calculations
ṁ_corn__in = ṅ_bushel__in * m_corn__bushel # lbm corn / hr
print(rf"ṁ_corn__in = {ṁ_corn__in:.0f} lbm/hr")

V̇_mash2 = 61875 gal/hr
ṅ_bushel__in = 2475 bushels/hr
ṁ_corn__in = 138600 lbm/hr


In [5]:
# from USDA statistics
ṅ_bushel__acre = 175 # bushels / acre / year

# annual bushel demand and acreage calculations
ṅ_bushel__annual = ṅ_bushel__in * T_uptime__annual # bushels / year

n_acres = ṅ_bushel__annual / ṅ_bushel__acre # acres
print(rf"n_acres ~= {round(n_acres,-3):.0f} acres")

n_acres ~= 112000 acres


### Problem 4 (Low)

> Calculate the molar flow rate $\left(\frac{lbmol}{hr}\right)$ of ethanol product.

#### Logic
The yield of 199-Proof Ethanol per bushel has been specified, and the bushel feed rate has already been calculated. The specifications for 199-Proof Ethanol include it's specific gravity (which we assume is relative to water at 4 C) and the weight content of Ethanol in 199-Proof Ethanol. Combining these allows us to calculate the yield of pure ethanol by mass. Dividing by ethanol's molecular weight gives us the molar flow rate of pure ethanol product.

#### Calculations

In [6]:
# 199-Proof Ethanol yield calculations
ρ_E199 = ρ_H2O__4C * SG_E199 # lbm E199 / gal E199
V̇_E199__out = V_E199__bushel * ṅ_bushel__in # gal E199 / hr
ṁ_E199__out = V̇_E199__out * ρ_E199 # lbm E199 / hr

# Pure Ethanol yield calculations 
ṁ_EtOH__out = ṁ_E199__out * Xwt_EtOH__E199 # lbm EtOH / hr
ṅ_EtOH__out = ṁ_EtOH__out / MW_EtOH # lbmol EtOH / hr

print(rf"ṅ_EtOH__out = {ṅ_EtOH__out:.2f} lbmol/hr")

ṅ_EtOH__out = 908.31 lbmol/hr


### Problem 5 (Low)
> How much water (combined fresh water and condensates) must be added to the mixing tank to obtain the desired mash volume?

#### Logic

Since we are specified the volume of mash coming out per bushel in, the specific gravity of the mash, and the mass of corn going in a trivial overall mass balance allows us to know how much mass is added (water and condensates) to the mash.

Simple mass balance on the mixer:

$$
\dot{m}_\text{corn,in} + \dot{m}_{H_2O,in} = \dot{m}_\text{mash1}
$$

Which then gives us:

$$
\dot{m}_{H_2O,in} = \dot{m}_\text{mash1} - \dot{m}_\text{corn,in}
$$

#### Calculations

In [7]:
# Mash calculations
V̇_mash1 = ṅ_bushel__in * V_mash1__bushel # gal mash1 / hr
ρ_mash1 = SG_mash1 * ρ_H2O__4C # lbm mash1 / gal
ṁ_mash1 = V̇_mash1 * ρ_mash1 # lbm mash1 / hr

# Water calculations
ṁ_H2O_in = ṁ_mash1 - ṁ_corn__in

print(rf"ṁ_H2O_in = {ṁ_H2O_in:.0f} lbm/hr")

ṁ_H2O_in = 360325 lbm/hr


### Problem 6 (Low)

> The $\text{CO}_2$ produced as a by-product from the fermentation reactor may be useful if it is recovered in sufficient quantity. Estimate the production rate of CO2 in $\frac{\text{lbmol}}{\text{hr}}$ and SCFM.

#### Logic
This is the formula for the relevant reaction:

$$
C_{12}H_{22}O_{11} + H_2O + (\text{yeast}) \rightarrow 4C_2H_5OH + 4CO_2 + (\text{yeast}) + (H_2O)
$$

Since the ethanol and $CO_2$ are produced in proportion to each other, and we don't start with any of either, their mole flow rates are also equal.

Since standard temperature and pressure (STP) aren't specified in the problem, we are using the STP defined in Zybooks (T = 0C, P = 1 atm), under which conditions $CO_2$ is a gas. Using the ideal gas law allows us to get a volumetric flow rate at STP.

In [8]:
ṅ_CO2__out = ṅ_EtOH__out # lbmol / hr
V̇_CO2_out = ṅ_CO2__out * R * T_R__STP / P_atm__STP / 60 # ft^3 / min at STP conditions, or SCFM

print(rf"ṅ_CO2__out = {ṅ_CO2__out:.1f} lbmol/hr")
print(rf"V̇_CO2_out = {V̇_CO2_out:.0f} SCFM (ft^3/min)")

ṅ_CO2__out = 908.3 lbmol/hr
V̇_CO2_out = 5435 SCFM (ft^3/min)


### Problem 7 (Medium)

> Estimate the temperature of the bottom streams leaving the azeotrope column and the stripper.

#### Logic

We know that the azeotrope column and the stripper are both operating at VLE. Because of this we can use Raoult's Law or just the Antoine equation 

#### Calculations

In [9]:
# knowns

x1 = Xmol_EtOH__E199 # ethanol
x2 = 1-x1 # water
P = 2.0265 # bar, system pressure in stripper

In [10]:
def psat(t,i):
    if i == 1:  # ethanol
        A = 8.13484	
        B = 1662.48	
        C = 238.131
     
    elif i == 2:  # water
        A = 8.05573	
        B = 1723.64	
        C = 233.076
     

        # Calculate saturation pressure using Antoine equation
    P_sat = 10 ** (A - (B/ (t + C))) / 750.062 # convert to bar
    return(P_sat)  # in bar

psat(100,2)

1.0132527754404501

In [11]:
# function for fsolve for stripper

def f(a):
    eq1 =  P - psat(a,2)
    return(eq1)

In [12]:
guesses = 160
f(guesses)

-4.219880124915525

In [13]:
ans = fsolve(f, guesses)
TF = ans[0] * 9/5 + 32

In [14]:
print('t =', round(ans[0], 4), 'Celsius')
print('t =', round(TF, 4), 'Farenheit')
# for the stripper

t = 120.572 Celsius
t = 249.0296 Farenheit


In [15]:
# function for fsolve for azeotrope column
Pac = 3.34373 # bar, system pressure for azeotrope column

def f1(a):
    t, y1 = a
    eq1 = x1 * psat(t,1) - y1 * Pac
    eq2 = x2 * psat(t,2) - (1-y1) * Pac
    return([eq1,eq2])

In [16]:
guesses1 = [200,0.8]
f1(guesses1)

[25.918410589219683, -0.3433648243825201]

In [17]:
ans1 = fsolve(f1, guesses1)
TF1 = ans1[0] * 9/5 + 32

In [18]:
print('t =', round(ans1[0], 4), 'Celsius')
print('t =', round(TF1, 4), 'Farenheit')
# for the azeotrope column

t = 113.2854 Celsius
t = 235.9137 Farenheit


### Problem 8 (High)

> What fraction of the dried grains must be recycled to control the moisture content of the grains entering the dryer? What production rate of dried grains can be expected from this process?
This problem involves solving multiple parts of the system. Start with figuring out the amount of dried grains leaving the ethanol concentrator.)

#### Logic

First we solved for the total amount of ethanol produced in the process. Because we are given the volumetric fractions of this stream, we are also able to solve for the amount of water and solids which we need later.  

All of the ethanol leaves the Ethanol Concentrator Column's Condenser as the tops. We are given the molar fractions of this stream leaving (AKA the Azetrope Column Inlet), so we can solve for the water leaving in the tops. Taking the differents of this tops leaving water and the total water, we can solve for the bottoms water flow rate.

The bottoms enters the flash containing the calculated bottoms water and all of the solids. The conditions of this stream(liquid + solids) are 50 psig and 305 F. This stream is flashed to atmospheric pressure producing and exiting vapor stream and a liquid stream that enters the centrifuge which spilts into two streams(solid-rich and water-rich). To solve for the liquid streams, we had to do an energy balance of the liquid water and solids entering the flash and the (stream 1) vapor water and (stream 2) liquid water and solids exiting using the follow equation solving for both m<sub>v</sub> and m<sub>l</sub>.

$$$$
$$ m_wh_w(305F, 64.7psi) + m_sh_s(305F) = m_vh_v(14.7psi, sat) + m_lh_l(14.7psi, sat) + m_sh_s(212F) $$
$$ m_w= m_v + m_l$$
$$$$
We can use a mass balance of the solids in the rest of the streams, plus the additons equations that the water leaving the flash (AKA entering the cenetrifuge) is equal to 0.70 of solid-rich streams and 0.96 of water-rich streams.

Solids for all of the streams using the fact that all of the solids entering must exit the whole dry grains process in the exit stream or return in the recyling thin-stillage stream, this allowed use to solve for the streams without having to solve the enthalpy of the two drying steps to calculated the water leaving mid-process.

We found that the fraction of dried grains recycled was 0.97887, and the flowrate of the producted dried grains is 41147.3563 lbm/hr.

#### Calculations

In [19]:
#PREPARATION RECOVERY SYSTEM
#ETHANOL BASIS

wtPer_Ethanol = 7.1/100 #wtfrac
wtPer_Solids = 6.9/100  #wtfrac
wtPer_Water = 1-wtPer_Ethanol-wtPer_Solids    #wtfrac

# lb_exitFlash*wtPer_Ethanol = lbEthanol
lb_exitFlash = ṁ_EtOH__out/wtPer_Ethanol #lbm/hr

lbSolids = wtPer_Solids*lb_exitFlash
lbWater = wtPer_Water*lb_exitFlash

#_______________________________________________

nEthConc_Dist = ṅ_EtOH__out/0.89
mEthConc_Dist_Water = nEthConc_Dist*0.11*MW_H2O
nEthConc_Dist_Water = nEthConc_Dist*0.11

mEthConc_Bottom_Water = lbWater - mEthConc_Dist_Water

#_______________________________________________

#Recycled Thin Stillage (AKA. mLV)
# "Thin stillage SG = 1.1"
# "The volume of the thin stillage added is 16% of the final mash volume going to the fermenters. "

SG_1_1 = 62.4*1.1   #lbm/ft3

vol_hr_mLV = 0.16*V̇_mash2/GAL_PER_FT3
lb_hr_mLV = vol_hr_mLV*SG_1_1

# "...thin stillage containing 4 wt% total solids..."
# ended up not using this, but calculated it later

lb_hr_mLV_Solids = lb_hr_mLV*0.04

# Entering water calculation leaving the flash
# SOLVED IN EXCEL AND ON iPad (written) need to move here

#FLASH ENERGY BALANCE
# inWater + inSolid = outVapor(water) + outLiquid(water) + outSolid
# In stream temperature = 305F and 50 psig
# Flash and Out 30 inHg (Iowa average atmospheric pressure)
# https://forecast.weather.gov/data/obhistory/KIOW.html

# ABSOLUTE PRESSURE ENTERING = 50psi + 30 inHg = (50 + 14.7346)psi = 64.7346 psi

# hWater_in STREAM TABLES (446.3293555kPa and 151.667 deg, saturated state, so h_fg)

hWater_in = 623.14   #kJ/kg
kJ_kj2BTU_lb = 2.326
hWater_in /= kJ_kj2BTU_lb   #BTU/lb

GivenEnthSolid = 0.25   #BTU/(lb*F)
hSolid_in = GivenEnthSolid*305 #BTU/lb

# 30inHg = 101.5914908kPa
# OUTS (Temp of Saturation = 100.0 degC) ***slight adjustments from table
# 100 degC = 212 F

hWater_L = 419.06 + 0.4 #kJ/kg
hWater_V = 2675.6 + 0.2 #kJ/kg

hWater_L /= kJ_kj2BTU_lb   #BTU/lb
hWater_V /= kJ_kj2BTU_lb   #BTU/lb

hSolid_out = GivenEnthSolid*212 #BTU/lb

# ENTERING FLASH
mEthConc_Bottom_Water
lbSolids

def f1(ms):
    mv, ml = ms
    mWi = mEthConc_Bottom_Water
    mSol = lbSolids
    e1 = mWi*hWater_in + mSol*(hSolid_in - hSolid_out) - (mv*hWater_V + ml*hWater_L)
    e2 = mWi - (mv + ml)
    return (e1, e2)


sol = fsolve(f1, (374747.19105377, 121123.61599223))
print(sol)

m_Flash_Water = sol[1] #lb/hr


def fx(var):
    m1, m2, m3, m4, m5, m6, m7 = var

    solids1 = 0.30*m1
    solids2 = 0.04*m2
    solidsLV = 0.04*lb_hr_mLV
    solids3 = 0.40*m3
    solids4 = (1-0.30)*m4
    solids5 = (1-0.10)*m5
    solids6 = (1-0.10)*m6
    solids7 = (1-0.10)*m7

    e1 = solidsLV + solids6 - lbSolids
    e2 = solids1 + solids2 - lbSolids
    e3 = solidsLV + solids3 - solids2
    e4 = solids1 + solids3 + solids7 - solids4
    e5 = solids6 + solids7 - solids5
    #e6 = solids1 + solids3 + solids7 - solids4  #SAME as e4!!!
    e6 = (1-0.30)*m1 + (1-0.04)*m2 - m_Flash_Water
    e7 = solids4 - solids5


    return (e1, e2, e3, e4, e5, e6, e7)

guess = (lb_hr_mLV*1.70, lb_hr_mLV*1.7, 20, lb_hr_mLV/10, lb_hr_mLV, lb_hr_mLV, lb_hr_mLV)
guess = (30000, 2000000, 600000, 120000, 100000, 40000, 6000000)

fx(guess)

solv = fsolve(fx, guess)
print(f"m1 = {solv[0]:.04f} lb/hr")
print(f"m2 = {solv[1]:.04f} lb/hr")
print(f"m3 = {solv[2]:.04f} lb/hr")
print(f"m4 = {solv[3]:.04f} lb/hr")
print(f"m5 = {solv[4]:.04f} lb/hr")
print(f"m6 = {solv[5]:.04f} lb/hr")
print(f"m7 = {solv[6]:.04f} lb/hr")
print()
print(f"The fraction of dry grains recycled is {solv[6]/solv[4]:.05f}")
print(f"And about = {solv[5]:.04f} lb/hr of dry grain producted")

[ 46545.93408642 458286.36369685]
m1 = 79646.7158 lb/hr
m2 = 419305.8986 lb/hr
m3 = 32846.5149 lb/hr
m4 = 2503610.0511 lb/hr
m5 = 1947252.2620 lb/hr
m6 = 41147.3563 lb/hr
m7 = 1906104.9056 lb/hr

The fraction of dry grains recycled is 0.97887
And about = 41147.3563 lb/hr of dry grain producted


### Problem 9 (High)

> What is the minimum pressure at which the cooking vessel must operate?; What is the pressure to which the mash must be flashed to provide a resultant liquor temperature of 145 °F?; At what rate (lbm/h) is water vaporized in this step (in the two flash units)?(You need to solve material and energy balances starting early in the process to determine the flowrate leaving the cooker.)

In [20]:
# minimum pressure at which cooking vessel must operate
# sat. pressure at 320 F = 160 C

def psat(t,i):
    if i == 2:  # water
        A = 8.05573	
        B = 1723.64	
        C = 233.076
     # Calculate saturation pressure using Antoine equation
    P_sat = 10 ** (A - (B/ (t + C))) / 750.062 # convert to bar
    return(P_sat)

min_pressure = psat(160, 2)
print('min_pressure =', round(min_pressure, 4), 'bar')

min_pressure = 6.2464 bar


In [21]:
# pressure to which mash must be flashed to provide a resultant liquor temp of 145 F = 62.7778 C

flash_pressure = psat(62.778, 2)
print('flash_pressure =', round(flash_pressure, 4), 'bar')

flash_pressure = 0.2263 bar


In [22]:
m_corn = 138600 # lbm/hr corn meal
m_h2o = 360533 # lbm/hr total water coming in to mixing tank

m_mash = m_corn + m_h2o
W_corn = m_corn/m_mash
W_h2o = m_h2o/m_mash

print(f"m_mash = {m_mash :.6f} lbm/hr flow rate of mash")
print(f"W_corn = {W_corn :.6f} weight % of corn solids in mash")
print(f"W_h2o = {W_h2o :.6f} weight % of water in mash")

m_mash = 499133.000000 lbm/hr flow rate of mash
W_corn = 0.277681 weight % of corn solids in mash
W_h2o = 0.722319 weight % of water in mash


In [23]:
# for precook vessel

pv_m_mashin = m_mash * W_corn # mass flowrate of mash solids into precook vessel
pv_m_h2oin = m_mash * W_h2o # mass flowrate of water into precook vessel
pv_m_mashout = pv_m_mashin # mass flowrate of mash solids OUT of precook vessel

tin_mash1 = 37.7778 # 100 F in C
tin_steam1 = 121 # in C, Tsat temp from 15 psig steam
tout_mash1 = 62.7778 # 145 F in C

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash1 = quad(Cpmash, tin_mash1, tout_mash1)[0]
delta_hmash_jPg1 = delta_hmash1/MWh2o
delta_hmash_jPlbm1 = delta_hmash_jPg1/lbmPg

print(f"delta_hmash1 = {delta_hmash_jPlbm1 :.6f} J/lbm = enthalpy change needed for mash")

delta_hmash1 = 47485.600905 J/lbm = enthalpy change needed for mash


In [24]:
# calculating delta hs and hs for energy balance around precook vessel


def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to precook vessel 
h_watermash_in1 = quad(Cph2o, ref, tin_mash1)[0]

# calculating h2, enthalpy of water vapor coming in to precook vessel
deltahvap = 39651 # @ 121 C, J/mol roughly from Dippr 
h_watervapor_in1 = quad(Cph2o, ref, tin_steam1)[0] + deltahvap

# calculating h3, enthalpy of water coming out of precook vessel in mash
h_watermash_out1 = quad(Cph2o, ref, tout_mash1)[0]


In [25]:
h_watermash_in_jPg1 = h_watermash_in1/MWh2o
h_watermash_in_jPlbm1 = h_watermash_in_jPg1/lbmPg

print(f"h_watermash_in_jPlbm1 = {h_watermash_in_jPlbm1 :.6f} J/lbm = enthalpy water coming in to precook vessel")

h_watervapor_in_jPg1 = h_watervapor_in1/MWh2o
h_watervapor_in_jPlbm1 = h_watervapor_in_jPg1/lbmPg

print(f"h_watervapor_in_jPlbm1 = {h_watervapor_in_jPlbm1 :.6f} J/lbm = enthalpy water vapor coming in precook vessel")

h_watermash_out_jPg1 = h_watermash_out1/MWh2o
h_watermash_out_jPlbm1 = h_watermash_out_jPg1/lbmPg

print(f"h_watermash_out_jPlbm1 = {h_watermash_out_jPlbm1 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm1 = 71756.061354 J/lbm = enthalpy water coming in to precook vessel
h_watervapor_in_jPlbm1 = 1228690.553192 J/lbm = enthalpy water vapor coming in precook vessel
h_watermash_out_jPlbm1 = 119241.662259 J/lbm = enthalpy water coming out in mash


In [26]:
# energy balance for precook vessel

def f_1(a):
    m_h2ovaporin1, m_h2omash_out1 = a # unknowns, water coming in as vapor and water coming out in mash
    
    eq1 = pv_m_h2oin + m_h2ovaporin1 - m_h2omash_out1
    eq2 = pv_m_mashin * delta_hmash_jPlbm1 + m_h2omash_out1 * h_watermash_out_jPlbm1 - m_h2ovaporin1 * h_watervapor_in_jPlbm1 - pv_m_h2oin * h_watermash_in_jPlbm1
    return([eq1,eq2])

guess_1 = [50000, 500000]
ans1 = fsolve(f_1, guess_1)
m_h2ovaporin1 = ans1[0]
m_h2omash_out1 = ans1[1]
print(f"m_h2ovaporin1 = {m_h2ovaporin1 :.6f} lbm/hr = flowrate of vapor in to precook Vessel")
print(f"m_h2omash_out1 = {m_h2omash_out1 :.6f} lbm/hr = flowrate of water in mash out of precook Vessel")


m_h2ovaporin1 = 21363.427040 lbm/hr = flowrate of vapor in to precook Vessel
m_h2omash_out1 = 381896.427040 lbm/hr = flowrate of water in mash out of precook Vessel


In [27]:
m_mashnew1 = pv_m_mashout + m_h2omash_out1  # total flowrate (mash + water) coming out of precook vessel
W_mash1 = pv_m_mashout/m_mashnew1
W_h2o1 = m_h2omash_out1/m_mashnew1

print(f"W_mash1 = {W_mash1 :.6f} new weight % of corn solids in mash out of precook vessel")
print(f"W_h2o1 = {W_h2o1 :.6f} new weight % of water in mash out of precook vessel")
print(f"m_mashnew1 = {m_mashnew1 :.6f} lbm/hr = flowrate of mash and water out of precook vessel")

W_mash1 = 0.266284 new weight % of corn solids in mash out of precook vessel
W_h2o1 = 0.733716 new weight % of water in mash out of precook vessel
m_mashnew1 = 520496.427040 lbm/hr = flowrate of mash and water out of precook vessel


In [28]:
# heat exchanger drives mash from 145 F to 230 F see question 10

In [29]:
# heater from 230 F to 320 F

# for heater 

heater_m_mashin = m_mashnew1 * W_mash1 # mass flowrate of mash solids into heater
heater_m_h2oin = m_mashnew1 * W_h2o1 # mass flowrate of water into heater
heater_m_mashout = heater_m_mashin # mass flowrate of mash solids OUT of heater

tin_mash2 = 110 # 230 F in C
tin_steam2 = 185 # in C, Tsat temp from 150 psig steam
tout_mash2 = 160 # 320 F in C

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash2 = quad(Cpmash, tin_mash2, tout_mash2)[0]
delta_hmash_jPg2 = delta_hmash2/MWh2o
delta_hmash_jPlbm2 = delta_hmash_jPg2/lbmPg

print(f"delta_hmash2 = {delta_hmash_jPlbm2 :.6f} J/lbm = enthalpy change needed for mash in heater")

delta_hmash2 = 94971.201809 J/lbm = enthalpy change needed for mash in heater


In [30]:
# calculating delta hs and hs for energy balance around heater


def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to heater
h_watermash_in2 = quad(Cph2o, ref, tin_mash2)[0]

# calculating h2, enthalpy of water vapor coming in to heater
deltahvap2 = 35939 # @ 185 C, J/mol roughly from Dippr
h_watervapor_in2 = quad(Cph2o, ref, tin_steam2)[0] + deltahvap2

# calculating h3, enthalpy of water coming out of heater in mash
h_watermash_out2 = quad(Cph2o, ref, tout_mash2)[0]


In [31]:
h_watermash_in_jPg2 = h_watermash_in2/MWh2o
h_watermash_in_jPlbm2 = h_watermash_in_jPg2/lbmPg

print(f"h_watermash_in_jPlbm2 = {h_watermash_in_jPlbm2 :.6f} J/lbm = enthalpy water coming in to heater")

h_watervapor_in_jPg2 = h_watervapor_in2/MWh2o
h_watervapor_in_jPlbm2 = h_watervapor_in_jPg2/lbmPg

print(f"h_watervapor_in_jPlbm2 = {h_watervapor_in_jPlbm2 :.6f} J/lbm = enthalpy water vapor coming in heater")

h_watermash_out_jPg2 = h_watermash_out2/MWh2o
h_watermash_out_jPlbm2 = h_watermash_out_jPg2/lbmPg

print(f"h_watermash_out_jPlbm2 = {h_watermash_out_jPlbm2 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm2 = 208936.643981 J/lbm = enthalpy water coming in to heater
h_watervapor_in_jPlbm2 = 1256743.585111 J/lbm = enthalpy water vapor coming in heater
h_watermash_out_jPlbm2 = 303907.845790 J/lbm = enthalpy water coming out in mash


In [32]:
# energy balance for heater

def f_2(a):
    m_h2ovaporin2, m_h2omash_out2 = a # unknowns, water coming in as vapor and water coming out in mash
    
    eq1 = heater_m_h2oin + m_h2ovaporin2 - m_h2omash_out2
    eq2 = heater_m_mashin * delta_hmash_jPlbm2 + m_h2omash_out2 * h_watermash_out_jPlbm2 - m_h2ovaporin2 * h_watervapor_in_jPlbm2 - heater_m_h2oin * h_watermash_in_jPlbm2
    return([eq1,eq2])

guess_2 = [50000, 500000]
ans2 = fsolve(f_2, guess_2)
m_h2ovaporin2 = ans2[0]
m_h2omash_out2 = ans2[1]
print(f"** m_h2ovaporin2 = {m_h2ovaporin2 :.6f} lbm/hr = flowrate of vapor (live steam) in to heater **")
print(f"m_h2omash_out2 = {m_h2omash_out2 :.6f} lbm/hr = flowrate of water in mash out of heater")


** m_h2ovaporin2 = 51879.006185 lbm/hr = flowrate of vapor (live steam) in to heater **
m_h2omash_out2 = 433775.433225 lbm/hr = flowrate of water in mash out of heater


In [33]:
m_mashnew2 = heater_m_mashout + m_h2omash_out2  # total flowrate (mash + water) coming out of heater
W_mash2 = heater_m_mashout/m_mashnew2
W_h2o2 = m_h2omash_out2/m_mashnew2

print(f"W_mash2 = {W_mash2 :.6f} new weight % of corn solids in mash out of heater")
print(f"W_h2o2 = {W_h2o2 :.6f} new weight % of water in mash out of heater")
print(f"m_mashnew2 = {m_mashnew2 :.6f} lbm/hr = flowrate of mash and water out of heater")

W_mash2 = 0.242149 new weight % of corn solids in mash out of heater
W_h2o2 = 0.757851 new weight % of water in mash out of heater
m_mashnew2 = 572375.433225 lbm/hr = flowrate of mash and water out of heater


In [34]:
# after cooking tank, flowrate is still consistent
# solving now around first flash tank at 15 psig

tin_mash3 = 160 # 320 F in C
tout_mash3 = 121 # in C, temp from 15 psig in flash. Used steam tables to figure it out
tout_steam3 = 121 # same as tout_mash3

flash_m_mashin = m_mashnew2 * W_mash2 # mass flowrate of mash solids into flash tank
flash_m_h2oin = m_mashnew2 * W_h2o2 # mass flowrate of water into flash tank
flash_m_mashout = flash_m_mashin # mass flowrate of mash solids OUT of flash tank

print(f"flash_m_h2oin = {flash_m_h2oin :.6f} lbm/hr = flowrate of water coming in to flash")
print(f"flash_m_mashin = {flash_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to flash")

flash_m_h2oin = 433775.433225 lbm/hr = flowrate of water coming in to flash
flash_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to flash


In [35]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash3 = quad(Cpmash, tin_mash3, tout_mash3)[0]
delta_hmash_jPg3 = delta_hmash3/MWh2o
delta_hmash_jPlbm3 = delta_hmash_jPg3/lbmPg

print(f"delta_hmash3 = {delta_hmash_jPlbm3 :.6f} J/lbm = enthalpy change of mash in first flash")

delta_hmash3 = -74077.537411 J/lbm = enthalpy change of mash in first flash


In [36]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to flash 
h_watermash_in3 = quad(Cph2o, ref, tin_mash3)[0]

# calculating h2, enthalpy of water coming out of flash in vapor
deltahvap3 = 39651 # @ 121 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_out3 = quad(Cph2o, ref, tout_steam3)[0] + deltahvap3

# calculating h3, enthalpy of water coming out in mash
h_watermash_out3 = quad(Cph2o, ref, tout_mash3)[0]



In [37]:
h_watermash_in_jPg3 = h_watermash_in3/MWh2o
h_watermash_in_jPlbm3 = h_watermash_in_jPg3/lbmPg

print(f"h_watermash_in_jPlbm3 = {h_watermash_in_jPlbm3 :.6f} J/lbm = enthalpy water coming in to flash")

h_watervapor_out_jPg3 = h_watervapor_out3/MWh2o
h_watervapor_out_jPlbm3 = h_watervapor_out_jPg3/lbmPg

print(f"h_watervapor_out_jPlbm3 = {h_watervapor_out_jPlbm3 :.6f} J/lbm = enthalpy water coming out in vapor")

h_watermash_out_jPg3 = h_watermash_out3/MWh2o
h_watermash_out_jPlbm3 = h_watermash_out_jPg3/lbmPg

print(f"h_watermash_out_jPlbm3 = {h_watermash_out_jPlbm3 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm3 = 303907.845790 J/lbm = enthalpy water coming in to flash
h_watervapor_out_jPlbm3 = 1228690.553192 J/lbm = enthalpy water coming out in vapor
h_watermash_out_jPlbm3 = 229830.308379 J/lbm = enthalpy water coming out in mash


In [38]:
def f_3(a):
    m_h2ovaporout3, m_h2omash_out3 = a # unknowns, water coming out as vapor and water coming out in mash
    
    eq1 = flash_m_h2oin - m_h2ovaporout3 - m_h2omash_out3
    eq2 = flash_m_mashin * delta_hmash_jPlbm3 + m_h2omash_out3 * h_watermash_out_jPlbm3 + m_h2ovaporout3 * h_watervapor_out_jPlbm3 - flash_m_h2oin * h_watermash_in_jPlbm3
    return([eq1,eq2])

guess_3 = [50000, 500000]
ans3 = fsolve(f_3, guess_3)
m_h2ovaporout3 = ans3[0]
m_h2omash_out3 = ans3[1]
print(f"** m_h2ovaporout3 = {m_h2ovaporout3 :.6f} lbm/hr = flowrate of vapor out of first flash **")
print(f"m_h2omash_out3 = {m_h2omash_out3 :.6f} lbm/hr = flowrate of water in mash out of first flash")

** m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash **
m_h2omash_out3 = 391326.889710 lbm/hr = flowrate of water in mash out of first flash


In [39]:
m_mashnew3 = flash_m_mashout + m_h2omash_out3  # total flowrate (mash + liq water) coming out of flash
W_mash3 = flash_m_mashout/m_mashnew3
W_h2o3 = m_h2omash_out3/m_mashnew3

print(f"W_mash3 = {W_mash3 :.6f} new weight % of corn solids in mash out of flash")
print(f"W_h2o3 = {W_h2o3 :.6f} new weight % of water in mash out of flash")
print(f"m_mashnew3 = {m_mashnew3 :.6f} lbm/hr = flowrate of mash and liq water out of flash")

W_mash3 = 0.261546 new weight % of corn solids in mash out of flash
W_h2o3 = 0.738454 new weight % of water in mash out of flash
m_mashnew3 = 529926.889710 lbm/hr = flowrate of mash and liq water out of flash


In [40]:
# solving now around flash vacuum tank at 0.2263 bar

tin_mash4 = 121 # in C
tout_mash4 = 62.78 # in C or 145 F, temp from 0.2263 bar in flash vacuum. Used steam tables to figure it out
tout_steam4 = 62.78 # or 145 F same as tout_mash4

flashvac_m_mashin = m_mashnew3 * W_mash3 # mass flowrate of mash solids into flash vacuum
flashvac_m_h2oin = m_mashnew3 * W_h2o3 # mass flowrate of water into flash vacuum
flashvac_m_mashout = flashvac_m_mashin # mass flowrate of mash solids OUT of flash vacuum

print(f"flashvac_m_h2oin = {flashvac_m_h2oin :.6f} lbm/hr = flowrate of water coming in to flash vacuum")
print(f"flashvac_m_mashin = {flashvac_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to flash vacuum")

flashvac_m_h2oin = 391326.889710 lbm/hr = flowrate of water coming in to flash vacuum
flashvac_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to flash vacuum


In [41]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmash4 = quad(Cpmash, tin_mash4, tout_mash4)[0]
delta_hmash_jPg4 = delta_hmash4/MWh2o
delta_hmash_jPlbm4 = delta_hmash_jPg4/lbmPg

print(f"delta_hmash4 = {delta_hmash_jPlbm4 :.6f} J/lbm = enthalpy change of mash in flash vacuum")

delta_hmash4 = -110584.467387 J/lbm = enthalpy change of mash in flash vacuum


In [42]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to flash vacuum
h_watermash_in4 = quad(Cph2o, ref, tin_mash4)[0]

# calculating h2, enthalpy of water coming out of flash vacuum in vapor
deltahvap4 = 42395 # @ 62.78 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_out4 = quad(Cph2o, ref, tout_steam4)[0] + deltahvap4

# calculating h3, enthalpy of water coming out in mash
h_watermash_out4 = quad(Cph2o, ref, tout_mash4)[0]


In [43]:
h_watermash_in_jPg4 = h_watermash_in4/MWh2o
h_watermash_in_jPlbm4 = h_watermash_in_jPg4/lbmPg

print(f"h_watermash_in_jPlbm4 = {h_watermash_in_jPlbm4 :.6f} J/lbm = enthalpy water coming in to flash vac")

h_watervapor_out_jPg4 = h_watervapor_out4/MWh2o
h_watervapor_out_jPlbm4 = h_watervapor_out_jPg4/lbmPg

print(f"h_watervapor_out_jPlbm4 = {h_watervapor_out_jPlbm4 :.6f} J/lbm = enthalpy water coming out of flash vac in vapor")

h_watermash_out_jPg4 = h_watermash_out4/MWh2o
h_watermash_out_jPlbm4 = h_watermash_out_jPg4/lbmPg

print(f"h_watermash_out_jPlbm4 = {h_watermash_out_jPlbm4 :.6f} J/lbm = enthalpy water coming out in mash")

h_watermash_in_jPlbm4 = 229830.308379 J/lbm = enthalpy water coming in to flash vac
h_watervapor_out_jPlbm4 = 1187231.013594 J/lbm = enthalpy water coming out of flash vac in vapor
h_watermash_out_jPlbm4 = 119245.840992 J/lbm = enthalpy water coming out in mash


In [44]:
def f_4(a):
    m_h2ovaporout4, m_h2omash_out4 = a # unknowns, water coming out as vapor and water coming out in mash
    
    eq1 = flashvac_m_h2oin - m_h2ovaporout4 - m_h2omash_out4
    eq2 = flashvac_m_mashin * delta_hmash_jPlbm4 + m_h2omash_out4 * h_watermash_out_jPlbm4 + m_h2ovaporout4 * h_watervapor_out_jPlbm4 - flashvac_m_h2oin * h_watermash_in_jPlbm4
    return([eq1,eq2])

guess_4 = [50000, 500000]
ans4 = fsolve(f_4, guess_4)
m_h2ovaporout4 = ans4[0]
m_h2omash_out4 = ans4[1]
print(f"** m_h2ovaporout4 = {m_h2ovaporout4 :.6f} lbm/hr = flowrate of vapor out of flash vac**")
print(f"m_h2omash_out4 = {m_h2omash_out4 :.6f} lbm/hr = flowrate of water in mash out of flash vac")

** m_h2ovaporout4 = 54871.251358 lbm/hr = flowrate of vapor out of flash vac**
m_h2omash_out4 = 336455.638352 lbm/hr = flowrate of water in mash out of flash vac


In [45]:
m_mashnew4 = flashvac_m_mashout + m_h2omash_out4  # total flowrate (mash + liq water) coming out of flash vacuum
W_mash4 = flashvac_m_mashout/m_mashnew4
W_h2o4 = m_h2omash_out4/m_mashnew4

print(f"W_mash4 = {W_mash4 :.6f} new weight % of corn solids in mash out of flash vac")
print(f"W_h2o4 = {W_h2o4 :.6f} new weight % of water in mash out of flash vac")
print(f"m_mashnew4 = {m_mashnew4 :.6f} lbm/hr = flowrate of mash and liq water out of flash vac")

W_mash4 = 0.291755 new weight % of corn solids in mash out of flash vac
W_h2o4 = 0.708245 new weight % of water in mash out of flash vac
m_mashnew4 = 475055.638352 lbm/hr = flowrate of mash and liq water out of flash vac


#### Answers

**min_pressure cooking vessel must operate = 6.2464 bar \
flash_pressure to provide resultant temp of 145 F = 0.2263 bar \
m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash \m_h2ovaporout4 = 54871.251358 lbm/hr = flowrate of vapor out of flash vac**

### Problem 10 (Medium)

> Determine the rate (lbm/h) at which 15-psig steam must be supplied to heat the mash from the precooking vessel to 230 °F. How much live steam must be injected to raise the temperature of this stream to 320 °F? It has been suggested that a portion of these steam requirements could be met by the vapor from the flash occurring at 15 psig. Is this feasible? If so, what fraction of the required 15-psig steam can be supplied by this means?

In [46]:
# going back to heat exchanger heating mash to 230 F

print(f"W_mash1 = {W_mash1 :.6f} new weight % of corn solids in mash out of precook vessel")
print(f"W_h2o1 = {W_h2o1 :.6f} new weight % of water in mash out of precook vessel")
print(f"m_mashnew1 = {m_mashnew1 :.6f} lbm/hr = flowrate of mash and water out of precook vessel")


W_mash1 = 0.266284 new weight % of corn solids in mash out of precook vessel
W_h2o1 = 0.733716 new weight % of water in mash out of precook vessel
m_mashnew1 = 520496.427040 lbm/hr = flowrate of mash and water out of precook vessel


In [47]:
tin_mashHE = 62.78 # in C, or 145 F, inlet temp of mash going into Heat Exchanger
tout_mashHE = 110 # in C or 230 F, outlet temp of mash
tin_out_steamHE = 121 # in C Tsat of water in 15 psig sat steam

HE_m_mashin = m_mashnew1 * W_mash1 # mass flowrate of mash solids into Heat Exchanger
HE_m_h2oin = m_mashnew1 * W_h2o1 # mass flowrate of water into Heat Exchanger
HE_m_mashout = HE_m_mashin # mass flowrate of mash solids OUT of Heat Exchanger

print(f"HE_m_h2oin = {HE_m_h2oin :.6f} lbm/hr = flowrate of water coming in to heat exchanger")
print(f"HE_m_mashin = {HE_m_mashin :.6f} lbm/hr = flowrate of mash solids coming in to heat exchanger")

HE_m_h2oin = 381896.427040 lbm/hr = flowrate of water coming in to heat exchanger
HE_m_mashin = 138600.000000 lbm/hr = flowrate of mash solids coming in to heat exchanger


In [48]:
# calculating delta hs and hs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmashHE = quad(Cpmash, tin_mashHE, tout_mashHE)[0]
delta_hmash_jPgHE = delta_hmashHE/MWh2o
delta_hmash_jPlbmHE = delta_hmash_jPgHE/lbmPg

print(f"delta_hmashHE = {delta_hmash_jPlbmHE :.6f} J/lbm = enthalpy change of mash in Heat Exchanger")

delta_hmashHE = 89690.802989 J/lbm = enthalpy change of mash in Heat Exchanger


In [49]:
def Cph2o(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hh2oHE = quad(Cph2o, tin_mashHE, tout_mashHE)[0]
delta_hh2o_jPgHE = delta_hh2oHE/MWh2o
delta_hh2o_jPlbmHE = delta_hh2o_jPgHE/lbmPg

print(f"delta_hh2oHE = {delta_hh2o_jPlbmHE :.6f} J/lbm = enthalpy change of h2o in mash in Heat Exchanger")

delta_hh2oHE = 89690.802989 J/lbm = enthalpy change of h2o in mash in Heat Exchanger


In [50]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h2w, enthalpy of water vapor coming in to Heat Exchanger
deltahvapHE = 39651 # @ 62.78 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_inHE = quad(Cph2o, ref, tin_out_steamHE)[0] + deltahvapHE

# calculating h3w, enthalpy of water coming out of Heat Exchanger sat liq
h_water_outHE = quad(Cph2o, ref, tin_out_steamHE)[0]

In [51]:
h_watervapor_inHE_jPg = h_watervapor_inHE/MWh2o
h_watervapor_inHE_jPlbm = h_watervapor_inHE_jPg/lbmPg

print(f"h_watervapor_inHE_jPlbm = {h_watervapor_inHE_jPlbm :.6f} J/lbm = enthalpy watervapor coming in to Heat Exchanger")

h_water_outHE_jPg = h_water_outHE/MWh2o
h_water_outHE_jPlbm = h_water_outHE_jPg/lbmPg

print(f"h_water_outHE_jPlbm = {h_water_outHE_jPlbm :.6f} J/lbm = enthalpy water coming out of Heat Exchanger")

deltah_satstreamHE = h_water_outHE_jPlbm - h_watervapor_inHE_jPlbm
print(f"deltah_satstreamHE = {deltah_satstreamHE :.6f} J/lbm = enthalpy change of sat stream of Heat Exchanger")

h_watervapor_inHE_jPlbm = 1228690.553192 J/lbm = enthalpy watervapor coming in to Heat Exchanger
h_water_outHE_jPlbm = 229830.308379 J/lbm = enthalpy water coming out of Heat Exchanger
deltah_satstreamHE = -998860.244813 J/lbm = enthalpy change of sat stream of Heat Exchanger


In [52]:
def f_HE(a):
    m_satstream = a # unknowns, sat stream of 15 psig steam becoming sat liquid after Heat Exchanger
    
    eq1 = HE_m_mashin * delta_hmash_jPlbmHE + HE_m_h2oin * delta_hh2o_jPlbmHE + m_satstream * deltah_satstreamHE 
    return eq1

guess_HE = 500000
ansHE = fsolve(f_HE, guess_HE)
m_satstream = ansHE[0]

print(f"** m_satstream = {m_satstream :.6f} lbm/hr = flowrate of sat steam to heat mash to 230 F**")

** m_satstream = 46737.011245 lbm/hr = flowrate of sat steam to heat mash to 230 F**


#### Answers

**m_satstream = 46737.011245 lbm/hr = flowrate of sat steam to heat mash from precooking vessel to 230 F \
m_h2ovaporin2 = 51879.006185 lbm/hr = flowrate of vapor (live 150 psig steam) in to heater to raise temp to 320 F \
m_h2ovaporout3 = 42448.543516 lbm/hr = flowrate of vapor out of first flash at 15 psig** 

Comparing the flowrates above, it is not feasible for the steam requirements to be met by the vapor from the flash occurring at 15 psig. There is not enough needed for the precooking vessel to be maintained at 230 F. The heater also needs 150 psig so that cannot use the vapor from the flash. If we were to still use the vapor from the flash to satisfy a portion of the requirements, it can supply 42448/46737 or around 91% of the sat steam needed. 

### Question 11 (Very High)

> At what rates must heat be removed to condense the vapors from each of the distillation columns and the stripper? Suppose cooling water at 85 °F is available and the temperature increase of this water is limited to 25 °F. At what rates must water be supplied to each condenser?
(You need to start by solving the material balances around the azeotrope and stripper columns including the decanter and associated streams. You should already know the balances around the ethanol concentrator from Problem 8.)

#### Logic

We calculated the total mass flow rates of the water necessary to cool each of the condensers from each distillation column and the stripper. We did a total mass balance on the whole system starting from the Azeotrope Column to the stripper. After finding the pertinent mass flow rates, we solved for each Q. In the first distillation column, we solved for total heat provided by steam, then did an energy balance to find the remaining heat necessary to cool the distillate in condenser 1. That number came out to deltah_condenser_DC1_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in DC1. 
We then took this number and divided it by the heat capacity of water from 85F to 110F which gave us a total mass flow rate of water necessary at **m_h2o_condenser_DC1 = 3238915.531229 lbm/hr = flowrate of water supplied to condenser in DC1**.

We did a similar process in Condensers 2 and three, using the deltahVaporization for each component to find each Q.

**m_h2o_condenser_AC = 1756969.877170 lbm/hr = flowrate of water supplied to condenser in AC \
m_h2o_condenser_Str = 77415.391062 lbm/hr = flowrate of water supplied to condenser in Stripper**

#### Calculations

In [53]:
# knowns for Distillation column, ethanol concentrator
# Ethanol, water and mash going in. All ethanol goes to the top stream, all solids go to the bottom stream. mixture of water in both streams.

m_eth_inDC1 = 41102.2 # Ethanol in, lbm/hr 
m_h2o_inDC1 = 497857.8 # Water in, lbm/hr
m_mash_inDC1 = 39944.4 # mash in, lbm/hr

m_h2o_bottomsDC1 = 495870 # Water out in bottoms stream, lbm/hr
m_mash_bottomsDC1 = 39944.4 # mash out in the bottoms stream, lbm/hr

m_eth_topDC1 = 41102.2 # Ethanol out in the top stream, lbm/hr
m_h2o_topDC1 = 1987 # h2o out in the top stream, lbm/hr

t_in_DC1 = 121.11 # 250 F in C
t_bottoms_DC1 = 135 # 305 F in C
t_top_DC1 = 121.11 # 250 F in C

In [54]:
# calculating Q for reboiler

Tsat_150psig = 185 # C, Tsat of 150 psig steam for reboiler
m_150psig = 110000 # lbm/hr steam flowrate

# calculating delta hs and hs (delta hvap essentially)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1w, enthalpy of water vapor coming in to Reboiler
deltahvap_reboiler = 35939  # @ 185 C, J/mol roughly from Dippr (positive since liquid to steam)
h_watervapor_inreboiler = quad(Cph2o, ref, Tsat_150psig)[0] + deltahvap_reboiler

# calculating h2w, enthalpy of water coming out of Heat Exchanger sat liq
h_water_outreboiler = quad(Cph2o, ref, Tsat_150psig)[0]

In [55]:
h_watervapor_inreboiler_jPg = h_watervapor_inreboiler/MWh2o
h_watervapor_inreboiler_jPlbm = h_watervapor_inreboiler_jPg/lbmPg

print(f"h_watervapor_inreboiler_jPlbm = {h_watervapor_inreboiler_jPlbm :.6f} J/lbm = enthalpy watervapor coming in to reboiler")

h_water_outreboiler_jPg = h_water_outreboiler/MWh2o
h_water_outreboiler_jPlbm = h_water_outreboiler_jPg/lbmPg

print(f"h_water_outreboiler_jPlbm = {h_water_outreboiler_jPlbm :.6f} J/lbm = enthalpy water coming out of reboiler")

deltah_satstreamreboiler = h_water_outreboiler_jPlbm - h_watervapor_inreboiler_jPlbm
print(f"deltah_satstreamreboiler = {deltah_satstreamreboiler :.6f} J/lbm = enthalpy change of sat stream of reboiler")

h_watervapor_inreboiler_jPlbm = 1256743.585111 J/lbm = enthalpy watervapor coming in to reboiler
h_water_outreboiler_jPlbm = 351393.446695 J/lbm = enthalpy water coming out of reboiler
deltah_satstreamreboiler = -905350.138416 J/lbm = enthalpy change of sat stream of reboiler


In [56]:
Qreboiler = m_150psig * deltah_satstreamreboiler

print(f"Qreboiler = {Qreboiler :.6f} J/hr = heat transferred by reboiler")

Qreboiler = -99588515225.773026 J/hr = heat transferred by reboiler


In [57]:
# calculating hs and deltahs

def Cpmash(t):
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWh2o = 18.011

lbmPg = 0.002204 #conversion

delta_hmashDC1 = quad(Cpmash, t_in_DC1, t_bottoms_DC1)[0]
delta_hmashDC1_jPgHE = delta_hmashDC1/MWh2o
delta_hmashDC1_jPlbmHE = delta_hmashDC1_jPgHE/lbmPg

print(f"delta_hmashDC1_jPlbmHE = {delta_hmashDC1_jPlbmHE :.6f} J/lbm = enthalpy change of mash in Distillation Column 1(ethanol conc)")

delta_hmashDC1_jPlbmHE = 26382.999863 J/lbm = enthalpy change of mash in Distillation Column 1(ethanol conc)


In [58]:
# calculating hs and deltahs

def Cpethanol(t):
    A = 9.839E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

MWeth = 46.07
lbmPg = 0.002204 #conversion

delta_hethDC1 = quad(Cpethanol, t_in_DC1, t_top_DC1)[0]
delta_hethDC1_jPgHE = delta_hethDC1/MWeth
delta_hethDC1_jPlbmHE = delta_hethDC1_jPgHE/lbmPg

print(f"delta_hethDC1_jPlbmHE = {delta_hethDC1_jPlbmHE :.6f} J/lbm = enthalpy change of ethanol in Distillation Column 1(ethanol conc)")

delta_hethDC1_jPlbmHE = 0.000000 J/lbm = enthalpy change of ethanol in Distillation Column 1(ethanol conc)


In [59]:
# calculating delta hs and hs

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

# I chose as my reference state enthalpy of 0 at 32 F or 0 C
ref = 0 # C

# calculating h1, enthalpy of water coming in to DC1
h_water_inDC1 = quad(Cph2o, ref, t_in_DC1)[0]

# calculating h2, enthalpy of water coming out of DC1 at the top
h_water_outtopDC1 = quad(Cph2o, ref, t_top_DC1)[0]

# calculating h3, enthalpy of water coming out of DC1 at the bottom
h_water_outbottomDC1 = quad(Cph2o, ref, t_bottoms_DC1)[0]


In [60]:
h_water_inDC1_jPg = h_water_inDC1/MWh2o
h_water_inDC1_jPlbm = h_water_inDC1_jPg/lbmPg

print(f"h_water_inDC1_jPlbm = {h_water_inDC1_jPlbm :.6f} J/lbm = enthalpy water coming in to DC1")

h_water_outtopDC1_jPg = h_water_outtopDC1/MWh2o
h_water_outtopDC1_jPlbm = h_water_outtopDC1_jPg/lbmPg

print(f"h_water_outtopDC1_jPlbm = {h_water_outtopDC1_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the top")

h_water_outbottomDC1_jPg = h_water_outbottomDC1/MWh2o
h_water_outbottomDC1_jPlbm = h_water_outbottomDC1_jPg/lbmPg

print(f"h_water_outbottomDC1_jPlbm = {h_water_outbottomDC1_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the bottom")

h_water_inDC1_jPlbm = 230039.245023 J/lbm = enthalpy water coming in to DC1
h_water_outtopDC1_jPlbm = 230039.245023 J/lbm = enthalpy water coming out of DC1 at the top
h_water_outbottomDC1_jPlbm = 256422.244885 J/lbm = enthalpy water coming out of DC1 at the bottom


In [61]:
def f_DC1(a):
    Qcondenser = a # unknowns, heat transferred in condenser
    
    eq1 = m_mash_bottomsDC1*delta_hmashDC1_jPlbmHE + m_eth_topDC1*delta_hethDC1_jPlbmHE + m_h2o_bottomsDC1*h_water_outbottomDC1_jPlbm + m_h2o_topDC1*h_water_outtopDC1_jPlbm - m_h2o_inDC1*h_water_inDC1_jPlbm + Qcondenser + Qreboiler 
    return eq1

guess_DC1 = 500000
ansDC1 = fsolve(f_DC1, guess_DC1)
QcondenserDC1 = ansDC1[0]

print(f"** QcondenserDC1 = {QcondenserDC1 :.6f} J/hr = heat transferred in condenser in DC1**")

** QcondenserDC1 = 85452308015.559418 J/hr = heat transferred in condenser in DC1**


In [62]:
# calculating delta h water in condenser in DC1 (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_DC1 = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_DC1_jPg = deltah_condenser_DC1/MWh2o
deltah_condenser_DC1_jPlbm = deltah_condenser_DC1_jPg/lbmPg

print(f"deltah_condenser_DC1_jPlbm = {deltah_condenser_DC1_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in DC1")

deltah_condenser_DC1_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in DC1


In [63]:
m_h2o_condenser_DC1 = QcondenserDC1 / deltah_condenser_DC1_jPlbm

print(f"** m_h2o_condenser_DC1 = {m_h2o_condenser_DC1 :.6f} lbm/hr = flowrate of water supplied to condenser in DC1**")

** m_h2o_condenser_DC1 = 3238915.531229 lbm/hr = flowrate of water supplied to condenser in DC1**


In [64]:
# solving around condenser of azeotrope column
# vapor pentane, ethanol, water coming in and all condensed to liquid

m_pent_in_condAC = 179386.53 # lbm/hr
m_ethanol_in_condAC = 15515.94 # lbm/hr
m_h2o_in_condAC = 12525.28 # lbm/hr

In [65]:
Tsat_condAC = 66.11 # 151 F in C

#deltahvaps in positive since they are going from steam to liquid

deltahvap_pent_condAC = -23849 # J/mol
deltahvap_ethanol_condAC = -40048 # J/mol
deltahvap_h2o_condAC = -42249 # J/mol

MWpent = 72.15 # g/mol
MWethanol = 46.07 # g/mol

In [66]:

deltahvap_pent_condAC_jPg = deltahvap_pent_condAC/MWpent
deltahvap_pent_condAC_jPlbm = deltahvap_pent_condAC_jPg/lbmPg

print(f"deltahvap_pent_condAC_jPlbm = {deltahvap_pent_condAC_jPlbm :.6f} J/lbm = enthalpy pent coming in to condenser for Azeotrope Column")

deltahvap_ethanol_condAC_jPg = deltahvap_ethanol_condAC/MWethanol
deltahvap_ethanol_condAC_jPlbm = deltahvap_ethanol_condAC_jPg/lbmPg

print(f"deltahvap_ethanol_condAC_jPlbm = {deltahvap_ethanol_condAC_jPlbm :.6f} J/lbm = enthalpy ethanol coming in to condenser for Azeotrope Column")

deltahvap_h2o_condAC_jPg = deltahvap_h2o_condAC/MWh2o
deltahvap_h2o_condAC_jPlbm = deltahvap_h2o_condAC_jPg/lbmPg

print(f"deltahvap_h2o_condAC_jPlbm = {deltahvap_h2o_condAC_jPlbm :.6f} J/lbm = enthalpy water coming out of DC1 at the bottom")

deltahvap_pent_condAC_jPlbm = -149976.166310 J/lbm = enthalpy pent coming in to condenser for Azeotrope Column
deltahvap_ethanol_condAC_jPlbm = -394412.826374 J/lbm = enthalpy ethanol coming in to condenser for Azeotrope Column
deltahvap_h2o_condAC_jPlbm = -1064307.242771 J/lbm = enthalpy water coming out of DC1 at the bottom


In [67]:
Qcondenser_AC = m_pent_in_condAC*deltahvap_pent_condAC_jPlbm + m_ethanol_in_condAC*deltahvap_ethanol_condAC_jPlbm + m_h2o_in_condAC*deltahvap_h2o_condAC_jPlbm 

print(f"Qcondenser_AC = {Qcondenser_AC :.6f} J/hr = heat transferred by stream in condenser of Azeotrope Column")

Qcondenser_AC = -46354136028.062134 J/hr = heat transferred by stream in condenser of Azeotrope Column


In [68]:
# calculating delta h water in condenser in Azeotrope Column (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_AC = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_AC_jPg = deltah_condenser_AC/MWh2o
deltah_condenser_AC_jPlbm = deltah_condenser_AC_jPg/lbmPg

print(f"deltah_condenser_AC_jPlbm = {deltah_condenser_AC_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in Azeotrope Column")

deltah_condenser_AC_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in Azeotrope Column


In [69]:
m_h2o_condenser_AC = -Qcondenser_AC / deltah_condenser_AC_jPlbm 

print(f"** m_h2o_condenser_AC = {m_h2o_condenser_AC :.6f} lbm/hr = flowrate of water supplied to condenser in AC**")

** m_h2o_condenser_AC = 1756969.877170 lbm/hr = flowrate of water supplied to condenser in AC**


In [70]:
# solving around Stripper, need to get actual numbers

Tsat_top = 97.78 # 208 F in C

m_h2o_topDC1 = 1987 # h2o out in the top stream of DC1, lbm/hr
m_h2o_inAC = m_h2o_topDC1
m_h2o_bottomproduct_AC = 341.7122 # lbm/hr

m_h2o_wasteout_Str = m_h2o_inAC - m_h2o_bottomproduct_AC # h2o waste in bottom stream of Stripper, lbm/hr

m_h2o_inStr = 2776.156538 # lbm/hr, water into stripper
m_ethanol_inStr = 2366.47769 # lbm/hr, ethanol into stripper 

m_h2o_topStr = m_h2o_inStr - m_h2o_wasteout_Str # h2o coming out in the boilup stream of stripper

print(f"m_h2o_topStr = {m_h2o_topStr :.4f} lbm/hr, h2o coming out in the boilup stream of stripper")
print(f"m_ethanol_inStr = {m_ethanol_inStr :.4f} lbm/hr, ethanol coming out in the boilup stream of stripper")

#deltahvaps in positive since they are going from steam to liquid
deltahvap_ethanol_condStr = -37764 # J/mol
deltahvap_h2o_condStr = -40800 # J/mol

m_h2o_topStr = 1130.8687 lbm/hr, h2o coming out in the boilup stream of stripper
m_ethanol_inStr = 2366.4777 lbm/hr, ethanol coming out in the boilup stream of stripper


In [71]:
deltahvap_ethanol_condStr_jPg = deltahvap_ethanol_condStr/MWethanol
deltahvap_ethanol_condStr_jPlbm = deltahvap_ethanol_condStr_jPg/lbmPg

print(f"deltahvap_ethanol_condStr_jPlbm = {deltahvap_ethanol_condStr_jPlbm :.6f} J/lbm = enthalpy ethanol coming in to condenser for Stripper Column")

deltahvap_h2o_condStr_jPg = deltahvap_h2o_condStr/MWh2o
deltahvap_h2o_condStr_jPlbm = deltahvap_h2o_condStr_jPg/lbmPg

print(f"deltahvap_h2o_condStr_jPlbm = {deltahvap_h2o_condStr_jPlbm :.6f} J/lbm = enthalpy water coming in to condenser for Stripper Column")

deltahvap_ethanol_condStr_jPlbm = -371918.846764 J/lbm = enthalpy ethanol coming in to condenser for Stripper Column
deltahvap_h2o_condStr_jPlbm = -1027805.048760 J/lbm = enthalpy water coming in to condenser for Stripper Column


In [72]:
Qcondenser_Stripper = m_h2o_topStr * deltahvap_h2o_condStr_jPlbm + m_ethanol_inStr * deltahvap_ethanol_condStr_jPlbm 

print(f"Qcondenser_Stripper = {Qcondenser_Stripper :.6f} J/hr = heat transferred by stream in condenser of Stripper")

Qcondenser_Stripper = -2042450251.758530 J/hr = heat transferred by stream in condenser of Stripper


In [73]:
# calculating delta h water in condenser in Stripper (liquid going from 85 to 110 F)

def Cph2o(t): # in liquid
    A = 7.540E-02;B = 0;C = 0;D = 0
    Cpm = A + B*t + C*t**2 + D*t**3
    Cpm*=1000
    return(Cpm) #returns J/mol * C

t_incooling_water =  29.44 # 85 F in C
t_outcooling_water = 43.33 # 110 F in C

deltah_condenser_Str = quad(Cph2o, t_incooling_water, t_outcooling_water)[0] 

deltah_condenser_Str_jPg = deltah_condenser_Str/MWh2o
deltah_condenser_Str_jPlbm = deltah_condenser_Str_jPg/lbmPg

print(f"deltah_condenser_Str_jPlbm = {deltah_condenser_Str_jPlbm :.6f} J/lbm = enthalpy change of water supplied to condenser in Stripper")

deltah_condenser_Str_jPlbm = 26382.999863 J/lbm = enthalpy change of water supplied to condenser in Stripper


In [74]:
m_h2o_condenser_Str = -Qcondenser_Stripper  / deltah_condenser_Str_jPlbm  

print(f"** m_h2o_condenser_Str = {m_h2o_condenser_Str :.6f} lbm/hr = flowrate of water supplied to condenser in Stripper**")

** m_h2o_condenser_Str = 77415.391062 lbm/hr = flowrate of water supplied to condenser in Stripper**


### Problem 20 (Medium)

> Engineering design is composed of much more than technical calculations. As defined by ABET, the accreditation board for engineering programs in the US, solutions must meet specified technical needs but also consider public health, safety, and welfare as well as global, cultural, social, environmental, and economic factors. Table 4 contains definitions, examples, and details—used by BYU Chemical Engineering—on each of these ABET-specified factors. 
Explain how each of the following factors affect your design/calculations/recommendations for the bioethanol plant. Make sure to address each category separately. If one is not applicable, state why. Extensive detail is not required, as you haven’t been trained in safety and environmental considerations. Future classes will help you understand each of these items in more detail, but critical thinking, common sense, and Google can help you make many reasonable decisions regarding these items even at this point of your career. Stretch yourself a little bit and be as thoughtful as possible within the time constraints of this project.
- A. Public Health
- B. Public Safety
- C. Public Welfare
- D. Global Factors
- E. Cultural Factors
- F. Social Factors
- G. Environmental Factors
- H. Economic Factors

#### A - Public Health

Our process creates a waste stream of water with trace amounts of ethanol and propane. We send this to a waste treatment facility to ensure it is properly taken care of. We also produce CO2 in our reaction. There is a possibility of needing to scrub it before putting it back into the atmosphere. Our reactions occur at high temperatures and pressures, it will also be important to make sure that we have correct safety protocols and all pertinent information is labeled in our safety data sheets (ethanol, and by-products)

#### B - Public Safety

Because of our many high pressure lines with high temps, it will be critical to do risk analysis frequently. Having a pipe or flash tank rupture at 150psi can cause serious injury to employees. The probability of these issues are low when all equipment is properly maintained and kept up to specifications. This will require frequent checks on our gauges, the integrity of all reactors, pipes, and other parts of the process.  We will ensure there are safety measures in place with emergency shut down options and that all employees are properly trained on process hazards and company procedures. This training will include what to do in case of an emergency and to develop a plan for evacuation in case of fire, or other problems.

Ethanol is a flammable liquid, so measures will be taken to use approved containers and to prevent storage containers and tanks from potential ignition. We must also consider proper ventilation for all indoor portions of the factory to prevent ethanol vapors from accumulating.  For any employee working in hazardous conditions, we will need to provide appropriate PPE.

#### C - Public Welfare

There are a few features we can include to help people feel positive about the plant. For pedestrians, we can make sure everything is painted an appropriate color to blend in. We can also make sure the plant is located in a geographically favorable position. This could include being downwind and downstream from any towns if applicable.  This would help with any pollution affecting people as well as any smell or CO2 placed into the air. 

Operations can be completed efficiently and at appropriate times of day to reduce light and noise pollution. We can pay our employees appropriately and competitively based on the industry standard with opportunities for growth within the company. As our efficiency increases and profit follows, we can pay and employ more people. For employees, training programs to increase technical skills would really help in moral and career advancement. Sourcing our corn from local agriculture promotes the local economy as well.

#### D - Global Factors

Some parts of the factory that are frequently consumed may come from different countries and it will be important to make sure we have a consistent supply chain. Planning ahead for delays as well as potential profit losses due to tariffs and other shipping issues will also be important. 

When exporting and selling our bioethanol, we need to make sure all country specific regulations regarding transportation of hazardous material is followed. Because the US has fairly strict environmental regulations, it will be integral to operate our plane at maximum efficiency to be competitive globally. Optimizing steam flow through the plant to recycle and reuse heat will be critical in accomplishing this. 

Our plant needs to operate 330 days a year and so employing people in varying time zones who can work remotely and accomplish various tasks would be beneficial to operating efficiency. We will also need to make sure that all technology has proper cybersecurity measures taken. I have read about too many oil/gas companies being hacked and key supply chains being shut down.

#### E - Cultural Factors

Some potential cultural issues we forsee include to following: If we use significant water resources, make sure that all local communities still have adequate and equitable access. We have the job of convincing the public that our plant will benefit their economy, provide jobs, and will not destroy the local environment. Plan operation delays, maintenance, and other schedules around major important holidays based on country and religious observances. Make sure that our plant is not constructed on any culturally significant locations. Our employee vacation policies should be appropriate for the location. All signs and instructions should be posted in multiple locations so all language barriers can be managed. Allow employees of various religious backgrounds time and space to practice their beliefs.

#### F - Social Factors

Similar to stated before, offering training programs to allow employees to build skills and advance in their careers is important for moral, and filling all positions at the company. Create a way for employees to get to work if they use public transportation or need to walk/bike. Monitor that all interactions between supervisors and employees are respectful. Offering flexible schedules and other initiatives can help workers with families manage their lives. Sponsoring local events is a great way to increase public perception and impact the community. We will also need to ensure we are in compliance with labor laws.

#### G - Environmental Factors
We can make sure that we are not overharvesting the local land and placing too much strain on local farmers. Waste management and correct disposal measures are essential to reducing our environmental impact. Our plant produces CO2 and also could potentially impact air quality, soil/water quality, and more. We can run impact studies and engineer ways to scrub our pollutants. As discussed above, being compliant with OSHA is also important as well as other local regulations.

Regarding our byproducts and leftover material, we can find a way to recycle it, use it for other purposes, and reduce trash created. We will watch as new regulations are proposed by the government and preemptively design ways to meet these criteria.

#### H - Economic Factors

We will have to ensure that our plant generates enough product that we can be profitable. It is great to design all these programs to help employees and the community, as well as helping the environment. However, if the plant cannot make money then all the employees will lose their job. Creating a constant and predictable revenue stream is important to long-term well-being of the company and the plant.

When designing the plant, to attract investors, we need to make sure the ROR on this product is sufficient to allow us to gather the funds to create the project. Managing growth projections and economic growth will be important as well. Many parts of this process will fluctuate in price including sale price of ethanol, purchase price of corn and other materials. It will be important to monitor this and try to prevent financial risks.

Our energy costs will be a large portion of our expenses, so designing a plant that can be as efficient as possible will save us money, and reduce the impact of increasing energy costs.